In [ ]:
# Downgrade Python to 3.10 and set it as default
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-dev python3.10-venv -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10
!python3 --version

!wget https://bootstrap.pypa.io/get-pip.py
!python3 get-pip.py

!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.1-Stable
%cd /content/so-vits-svc

%pip install --upgrade pip setuptools

%pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118


!pip uninstall fairseq omegaconf hydra-core -y

!pip install pip==23.3.1

!pip cache purge

!pip install fairseq==0.12.2

!pip install faiss-cpu
!pip install torch==2.0.0+cu118 torchaudio==2.0.0+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib
!pip install "numpy<2.0"
!pip install soundfile
!pip install librosa
!pip install loguru



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

sovits_data_dir = "/content/drive/MyDrive/sovits4data/"
RAW_DIR = sovits_data_dir + "raw/"
RESULTS_DIR = sovits_data_dir + "results/"
FILELISTS_DIR = sovits_data_dir + "filelists/"
CONFIGS_DIR = sovits_data_dir + "configs/"
LOGS_DIR = sovits_data_dir + "logs/44k/"

sync_raw_and_results = True
if sync_raw_and_results:
  !mkdir -p {RAW_DIR}
  !mkdir -p {RESULTS_DIR}
  !rm -rf /content/so-vits-svc/raw
  !rm -rf /content/so-vits-svc/results
  !ln -s {RAW_DIR} /content/so-vits-svc/raw
  !ln -s {RESULTS_DIR} /content/so-vits-svc/results

sync_configs_and_logs = True
if sync_configs_and_logs:
  !mkdir -p {FILELISTS_DIR}
  !mkdir -p {CONFIGS_DIR}
  !mkdir -p {LOGS_DIR}
  !rm -rf /content/so-vits-svc/filelists
  !rm -rf /content/so-vits-svc/configs
  !rm -rf /content/so-vits-svc/logs/44k
  !ln -s {FILELISTS_DIR} /content/so-vits-svc/filelists
  !ln -s {CONFIGS_DIR} /content/so-vits-svc/configs
  !ln -s {LOGS_DIR} /content/so-vits-svc/logs/44k

In [ ]:
download_pretrained_model = True
D_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_D_320000.pth"
G_0_URL = "https://huggingface.co/datasets/ms903/sovits4.0-768vec-layer12/resolve/main/sovits_768l12_pre_large_320k/clean_G_320000.pth"

%cd /content/so-vits-svc

if download_pretrained_model:
    !curl -L {D_0_URL} -o logs/44k/D_0.pth
    !md5sum logs/44k/D_0.pth
    !curl -L {G_0_URL} -o logs/44k/G_0.pth
    !md5sum logs/44k/G_0.pth

In [ ]:
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"
zip_filename = "viveknewer.zip"
ZIP_PATH = sovits_data_dir + zip_filename

!unzip -od /content/so-vits-svc/dataset_raw {ZIP_PATH}

In [ ]:
%cd /content/so-vits-svc
!python resample.py

In [ ]:
%cd /content/so-vits-svc

speech_encoder = "vec768l12"
use_vol_aug = False
vol_aug = "--vol_aug" if use_vol_aug else ""

from pretrain.meta import download_dict
download_dict = download_dict()

url = download_dict[speech_encoder]["url"]
output = download_dict[speech_encoder]["output"]

import os
if not os.path.exists(output):
  !curl -L {url} -o {output}
  !md5sum {output}

!python preprocess_flist_config.py --speech_encoder={speech_encoder} {vol_aug}

In [ ]:
import os
os.environ['MPLBACKEND'] = 'Agg'

%cd /content/so-vits-svc

f0_predictor = "rmvpe"
use_diff = True

os.makedirs("pretrain", exist_ok=True)

if not os.path.exists("./pretrain/rmvpe.pt"):
    !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

# Download NSF HiFi-GAN if --use_diff is enabled
if use_diff:
    diff_param = "--use_diff"
    nsf_model_path = "./pretrain/nsf_hifigan/model"
    if not os.path.exists(nsf_model_path):
        !curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o nsf_hifigan_20221211.zip
        !unzip nsf_hifigan_20221211.zip
        !rm -rf pretrain/nsf_hifigan
        !mv -v nsf_hifigan pretrain
else:
    diff_param = ""

!python preprocess_hubert_f0.py --f0_predictor={f0_predictor} {diff_param}


In [ ]:
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"
CONFIG = sovits_data_dir + "configs/"
FILELISTS = sovits_data_dir + "filelists/"
DATASET = sovits_data_dir + "dataset.zip"

!cp -vr {CONFIG} /content/so-vits-svc/
!cp -vr {FILELISTS} /content/so-vits-svc/
!unzip {DATASET} -d /

In [ ]:
%cd /content/so-vits-svc

config_path = "configs/config.json"

from pretrain.meta import get_speech_encoder
url, output = get_speech_encoder(config_path)

import os
if not os.path.exists(output):
    !curl -L {url} -o {output}

!python train.py -c {config_path} -m 44k


In [ ]:
%cd /content/so-vits-svc
!curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o /content/so-vits-svc/nsf_hifigan_20221211.zip
!unzip nsf_hifigan_20221211.zip
!rm -rf pretrain/nsf_hifigan
!mv -v nsf_hifigan pretrain


In [ ]:
%cd /content/so-vits-svc
%run wav_upload.py --type audio

In [ ]:
wav_filename = "YourWAVFile.wav"  #@param {type:"string"}
model_filename = "G_210000.pth"  #@param {type:"string"}
model_path = "/content/so-vits-svc/logs/44k/" + model_filename
speaker = "viveknewer"  #@param {type:"string"}
trans = "0"  #@param {type:"string"}
cluster_infer_ratio = "0"  #@param {type:"string"}
auto_predict_f0 = False  #@param {type:"boolean"}
apf = ""
if auto_predict_f0:
  apf = " -a "

f0_predictor = "rmvpe" #@param ["crepe", "pm", "dio", "harvest", "rmvpe", "fcpe"]

enhance = False  #@param {type:"boolean"}
ehc = ""
if enhance:
  ehc = " -eh "
#@markdown

#@markdown Generally keep default:
config_filename = "config.json"  #@param {type:"string"}
config_path = "/content/so-vits-svc/configs/" + config_filename

from pretrain.meta import get_speech_encoder
url, output = get_speech_encoder(config_path)

import os

if f0_predictor == "rmvpe" and not os.path.exists("./pretrain/rmvpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/rmvpe.pt -o pretrain/rmvpe.pt

if f0_predictor == "fcpe" and not os.path.exists("./pretrain/fcpe.pt"):
  !curl -L https://huggingface.co/datasets/ylzz1997/rmvpe_pretrain_model/resolve/main/fcpe.pt -o pretrain/fcpe.pt

if not os.path.exists(output):
  !curl -L {url} -o {output}

kmeans_filenname = "kmeans_10000.pt"  #@param {type:"string"}
kmeans_path = "/content/so-vits-svc/logs/44k/" + kmeans_filenname
slice_db = "-40"  #@param {type:"string"}
wav_format = "flac"  #@param {type:"string"}

key = "auto" if auto_predict_f0 else f"{trans}key"
cluster_name = "" if cluster_infer_ratio == "0" else f"_{cluster_infer_ratio}"
isdiffusion = "sovits"
wav_output = f"/content/so-vits-svc/results/{wav_filename}_{key}_{speaker}{cluster_name}_{isdiffusion}_{f0_predictor}.{wav_format}"

%cd /content/so-vits-svc
!python inference_main.py -n {wav_filename} -m {model_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -c {config_path} -cm {kmeans_path} -sd {slice_db} -wf {wav_format} {apf} --f0_predictor={f0_predictor} {ehc}

#@markdown

#@markdown If you dont want to download from here, uncheck this.
download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)